In [22]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import sklearn
import scipy
import numpy as np
import matplotlib.pyplot as plt
from html_sanitizer import Sanitizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
import random as rnd
from sklearn.metrics import f1_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
\
from nltk.stem import WordNetLemmatizer
import bs4
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.ensemble import RandomForestClassifier
import xgboost
from sklearn.preprocessing import normalize
from sklearn.neighbors import KNeighborsClassifier

In [23]:
data = pd.read_csv("TitlesTrain.csv")
data_1 = pd.read_csv("final_parse.csv")
y_train = np.array(data["target"])
groups_train = np.array(data["group"])
texts = np.array(data["information"])
texts_1 = np.array(data_1["word"])

In [24]:
data = pd.merge(data, data_1, left_on='doc_id', right_on='doc_id',how="inner")

In [25]:
# texts = []
# for i in range(data.shape[0]):
#     #print(data.iloc[i]["information"],data.iloc[i]["word"])
#     texts.append(str(data.iloc[i]["information"]) + " " + str(data.iloc[i]["word"]))

In [26]:
data.head()

,Unnamed: 0_x,information,group,target,doc_id,Unnamed: 0_y,word
0,0,ваз 21213 замена подшипник ступица нива \n,1,0,15731,15730,тормозной приложение трансмиссия общий пружина...
1,1,ваз 2107 опт сочи сравнивать цена купить п...,1,0,14829,14828,питание электрооборудование prestige niva кпп ...
2,2,купить ступица лад калина2 трансмиссия пере...,1,0,15764,15763,весь аксессуар акпп цена хонда автопутешествие...
3,3,классика 21010 21074 \n,1,0,17669,17668,center learn models template templates website...
4,4,ступица нива замена подшипник свой рука \n,1,0,14852,14851,амортизация амортизатор безопасность ваз ваш в...


In [27]:
p = y_train.mean()

In [28]:
for i in range(len(texts)):
    if texts[i] is np.nan:
        texts[i] = ""

In [29]:
new_texts = np.array(new_texts)

In [30]:
vect = TfidfVectorizer()
X_train = vect.fit_transform(np.append(texts, new_texts))

In [31]:
texts.shape, new_texts.shape

((11690,), (16627,))

In [32]:
from sklearn import (manifold, decomposition)

In [33]:
X_train.shape

(28317, 27473)

In [34]:
X_train.shape, groups_train.shape

((28317, 27473), (11690,))

In [98]:

neigh = 25
XTrain = []
for i in np.unique(groups_train):
    X_ = X_train[groups_train == i]
    
    distMart = sklearn.metrics.pairwise_distances(X_, metric="cosine")
    #print(distMart)
    meanDist = np.sort(distMart, axis=1)[:,4:neigh]
    lis = []
    meanD = np.sort(distMart, axis=1)[:,:].mean(axis=1)
    stdD = np.sort(distMart, axis=1)[:,:].std(axis=1)
    
    
    #DIST = sklearn.metrics.pairwise.cosine_distances(X_,  X_train[-129 - 1 + i, :])
    #print(distMart)
    #print(meanD.shape)
    for i1 in range(meanDist.shape[0]):
        rr = [j for j in meanDist[i1,:]]
        rr.append(meanD[i1])
        rr.append(stdD[i1])
        
        #rr.append(DIST[i1])
        XTrain.append(rr)
        #print(i.shape)
    #print(i)
    
    

    
#XTrain = np.array(XTrain)
    


In [99]:
XTrain = np.array(XTrain)

In [100]:
XTrain.shape

(11690, 23)

In [101]:
with open("DBSCAN_VECTOR.txt", "r") as f:
    lis = [int(i) for i in f.read().split()]
    #print(lis)
    XTrain = np.append(XTrain, np.array(lis).reshape((-1,1)), axis=1)

In [102]:
# with open("KMeans_matr.txt", "r") as f:
#     lis = [int(i) for i in f.read().split()]
#     lis = np.array(lis).reshape((-1,23))  
#     print(lis.shape, XTrain.shape)
#     XTrain = np.append(XTrain, lis, axis=1)

In [103]:
# scl = StandardScaler()
# scl.fit(XTrain)
# XTrain = scl.transform(XTrain)

In [104]:
XTrain.shape, y_train.shape

((11690, 24), (11690,))

In [105]:
XTrain.shape

(11690, 24)

In [106]:
def make_big_matr(matr):
    matr1 = matr[:,:]
    for i in range(matr.shape[1]):
        for j in range(i, matr.shape[1]):
                #print((matr[:,i] * matr[:,j]).shape, matr1.shape)
                matr1 = np.concatenate((matr1, matr[:,i].reshape((-1,1)) * matr[:,j].reshape((-1,1))), axis=1)
    return matr1

In [107]:
#XTrain = make_big_matr(XTrain)

In [108]:
XTrain[:texts.shape[0],:].shape

(11690, 24)

In [109]:
X_ = decomposition.TruncatedSVD(n_components=50).fit_transform(X_train)
XTrain = np.append(XTrain,X_[:texts.shape[0],:], axis=1)



In [110]:
big_texts = data["word"]

In [111]:
for i in range(len(big_texts)):
    if big_texts[i] is np.nan:
        big_texts[i] = ""

In [112]:
# res1 = []
# res2 = []
# for i in np.unique(groups_train):
#     X_ = X_train[groups_train == i]
#     tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
#     X_ = tsne.fit_transform(X_.toarray())
#     res1 += list(X_[:,0])
#     res2 += list(X_[:,1])
#     print(i)

In [113]:
XTrain.shape

(11690, 74)

In [114]:
# XTrain = np.append(XTrain, np.array(res2).reshape((-1,1)), axis=1)
# XTrain = np.append(XTrain, np.array(res1).reshape((-1,1)), axis=1)

In [115]:

folds = 8
mean_roc_auc = 0
mean_f1 = 0
for train_index, test_index in GroupKFold(n_splits=folds).split(XTrain, y_train, groups_train):
    
    xTrain = -XTrain[train_index]
    yTrain = y_train[train_index]
    
    xTest = -XTrain[test_index]
    yTest = y_train[test_index]
    clf = RandomForestClassifier(n_estimators=2000, 
                                 min_samples_leaf=2,
                                 min_samples_split=4,
                                 max_features="log2",
                                 criterion="entropy",
                                 max_depth=15,
                                 n_jobs=-1,
                                 class_weight="balanced"
                                    
                                )
    clf.fit(xTrain, yTrain)
    print("\there")
    res = clf.predict(xTrain)
    print(roc_auc_score(yTrain, res))
    res = clf.predict(xTest)
    print(roc_auc_score(yTest, res))
    
    a1 = clf.predict(xTest)
    threeshold = np.quantile(a1, 1-p)
    
    mean_roc_auc += roc_auc_score(yTest, a1)
    #print(a1, threeshold)
    #a1 = (a1 > threeshold).astype(int)
    print(np.unique(a1,return_counts=True))
    print("F1 ", f1_score(yTest, a1))
    mean_f1 += f1_score(yTest, a1)


print(mean_roc_auc / folds)
print(mean_f1 / folds)
    

	here
0.9788321105075061
0.8812906943477846
(array([0, 1]), array([947, 505]))
F1  0.8546845124282983
	here
0.9782040716410774
0.8350070859600255
(array([0, 1]), array([990, 467]))
F1  0.731995277449823
	here
0.9816824995120212
0.790192566054635
(array([0, 1]), array([1027,  435]))
F1  0.6920332936979785
	here
0.9780213621410124
0.8190687453253325
(array([0, 1]), array([1100,  362]))
F1  0.736842105263158
	here
0.9790693370254728
0.8547022135257429
(array([0, 1]), array([1047,  428]))
F1  0.7619047619047619
	here
0.9778901829339851
0.8134741643437997
(array([0, 1]), array([1036,  425]))
F1  0.730952380952381
	here
0.9774087607420174
0.8678169602388867
(array([0, 1]), array([1001,  458]))
F1  0.8124306326304107
	here
0.9815766813223332
0.8070325492991208
(array([0, 1]), array([1043,  419]))
F1  0.7281323877068558
0.833573122386916
0.7561219190042083


In [72]:
print("F1 ", f1_score(y_train, res))

F1  0.7844418052256532


In [117]:
all_titles = pd.read_csv("docs_titles.tsv", sep="\t")
test_groups = pd.read_csv("test_groups.csv")
table = pd.merge(test_groups, all_titles, left_on='doc_id', right_on='doc_id',how="left")
table.shape

(16627, 4)

In [118]:
table.head()

,pair_id,group_id,doc_id,title
0,11691,130,6710,КАК ПРОПИСАТЬ АДМИНКУ В КС 1.6 СЕБЕ ИЛИ ДРУГУ ...
1,11692,130,4030,Скачать: SGL-RP доработка | Слив мода [MySQL] ...
2,11693,130,5561,Как прописать админку в кс 1.6 - Counter-Strik...
3,11694,130,4055,Как прописать простую админку в кс 1 6
4,11695,130,4247,Подбор админов для сервера по КОД_4 [Архив] -...


In [119]:
groups_test = np.array(table["group_id"])
texts = np.array(table["title"])
for i in range(len(texts)):
    if texts[i] is np.nan:
        texts[i] = ""

In [120]:
mystem = Mystem() 
rus_stopwords = stopwords.words("russian")
eng_stopwords = stopwords.words("english")
html_stopwords = ["span", "br", "a", "href", "img", "www", "com", "google", "ru", "html", "http", "https"]
stemming("Привет, медвед я в доме у  жен")

'привет  медведы   дом   жена\n'

In [121]:
def delete_symbols(text):
    res = ""
    text = text.lower()
    
    for i in text:
        t = i    
        if not(
                (t >= '0' and t <= '9') or 
                (t >= 'a' and t <= 'z') or
                (t >= 'а' and t <= 'я')
              ):
            t = ' '
            
        res += t
        
    return res 


def list2string(myList):  
    myStr = ""  
    
    for word in myList:
        if (word not in rus_stopwords) and (word not in eng_stopwords ) and (word not in html_stopwords):
            myStr += word
    
    return myStr

def stemming(text):
    text = delete_symbols(text)
    
    tokens = mystem.lemmatize(text)
    stri = list2string(tokens)
    #print(stri)
    return stri
    

In [122]:
new_texts = []
for i in texts:
    new_texts.append(stemming(i))

In [123]:
vect = TfidfVectorizer()
X_test = vect.fit_transform(new_texts)

In [124]:
pairs_id = []
groups_id = []

In [125]:

neigh = 25
XTest = []
for i in np.unique(groups_test):
    X_ = X_train[np.append(groups_train, groups_test) == i]
    
    distMart = sklearn.metrics.pairwise_distances(X_, metric="cosine")
    #print(distMart)
    meanDist = np.sort(distMart, axis=1)[:,4:neigh]
    lis = []
    meanD = np.sort(distMart, axis=1)[:,:].mean(axis=1)
    stdD = np.sort(distMart, axis=1)[:,:].std(axis=1)
    for i in range(meanDist.shape[0]):
        rr = [j for j in meanDist[i,:]]
        rr.append(meanD[i])
        rr.append(stdD[i])
        XTest.append(rr)
        #print(i.shape)
        
    

    
#XTrain = np.array(XTrain)
    


In [126]:
XTest = np.array(XTest)

In [127]:
XTest.shape

(16627, 23)

In [128]:
XTrain.shape

(11690, 74)

In [129]:
with open("TEST_DBSCAN_VECTOR.txt", "r") as f:
    lis = [int(i) for i in f.read().split()]
    print(len(lis),XTest.shape)
    XTest = np.append(XTest, np.array(lis).reshape((-1,1)), axis=1)

16627 (16627, 23)


In [141]:
X_ = decomposition.TruncatedSVD(n_components=50).fit_transform(X_train)
XTest = np.append(XTest,X_[11690:,:], axis=1)

In [137]:
XTest.shape

(16627, 24)

In [384]:
res1 = []
res2 = []
for i in np.unique(groups_test):
    X_ = X_test[groups_test == i]
    tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
    X_ = tsne.fit_transform(X_.toarray())
    res1 += list(X_[:,0])
    res2 += list(X_[:,1])
    print(i)

130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309


In [385]:
XTest = np.append(XTest, np.array(res2).reshape((-1,1)), axis=1)
XTest = np.append(XTest, np.array(res1).reshape((-1,1)), axis=1)

In [142]:
XTest.shape, XTrain.shape

((16627, 74), (11690, 74))

In [83]:
groups_train

array([  1,   1,   1, ..., 129, 129, 129])

In [87]:
y_test = np.array(pd.read_csv("HAHA.csv")["target"])

In [143]:
mean_f1 = 0
result = []
folds = 8
for train_index, test_index in GroupKFold(n_splits=folds).split(X_train[:11690,:], y_train, groups_train):
    #threeshold = 0.1
    xTrain = XTrain[train_index]
    yTrain = y_train[train_index]
    
    xTest = XTrain[test_index]
    yTest = y_train[test_index]
    
    clf = RandomForestClassifier(n_estimators=2000, 
                                 min_samples_leaf=2,
                                 min_samples_split=4,
                                 max_features="log2",
                                 criterion="entropy",
                                 max_depth=15,
                                 n_jobs=-1,
                                 class_weight="balanced"
                                    
                                )
    clf.fit(xTrain, yTrain)
    a1 = clf.predict(XTest)
    
    
    result.append(a1)
    a1 = clf.predict(xTest)
    print("F1 ", f1_score(yTest, a1))
    mean_f1 += f1_score(yTest, a1)
    

F1  0.8516746411483254
F1  0.731535756154748
F1  0.6848989298454221
F1  0.7351351351351351
F1  0.7603513174404015
F1  0.736094674556213
F1  0.8106312292358804
F1  0.7199046483909416


In [144]:
r

NameError: name 'r' is not defined

In [145]:
r = np.array(result)
r = r.sum(axis=0)

In [146]:
np.unique(r, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8]),
 array([11689,   161,    97,    57,    72,    64,    92,   175,  4220]))

In [147]:
np.unique(r, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8]),
 array([11689,   161,    97,    57,    72,    64,    92,   175,  4220]))

In [148]:

for i in range(r.shape[0]):
    if r[i] > 1:
        r[i] = 1
    else:
        r[i] = 0

In [149]:
r = np.array(r)
r.mean(), p

(0.2873037830035484, 0.28751069289991443)

In [151]:
with open("HAHA.csv", "w") as f:
    print("pair_id,target", file=f)
    for i in range(len(r)):
        print(table["pair_id"].iloc[i], ",", r[i], file=f, sep="")

In [113]:
XTrain.shape

(11690, 24)